Let's start by some imports: stats and basic numerical operations.

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn import preprocessing

The data lives in a csv file. Let's read that in a pandas data frame:

In [2]:
file_data = 'data_files/Csub_tsnr_time_motion.csv'
data = pd.read_csv(file_data)
data

,Time,Scan_dates,Visit,Site,id,tSNR_raw,Scanner,FD,FD_scrubbed,Total_volume,Volumes_scrubbed,Volumes_remaining
0,44,7/3/14,1,CHUM,CHUM1,100.891228,Philips,0.184465,0.182316,297,4,293
1,917,11/22/16,2,CHUM,CHUM2,85.067749,Philips,0.249586,0.225795,297,42,255
2,156,10/23/14,1,CHUS,CHUS1,79.841675,Philips,0.177949,0.174218,297,8,289
3,337,4/22/15,2,CHUS,CHUS2,78.079979,Philips,0.246182,0.213796,297,91,206
4,898,11/3/16,3,CHUS,CHUS3,77.908920,Philips,0.251286,0.232445,297,34,263
5,22,6/11/14,1,CINQ,CINQ1,90.081963,Philips,0.289165,0.246616,297,86,211
6,325,4/10/15,2,CINQ,CINQ2,94.104416,Philips,0.301787,0.208972,297,118,179
7,911,11/16/16,3,CINQ,CINQ3,90.639122,Philips,0.217974,0.183525,297,41,256
8,924,11/29/16,1,EDM,EDM1,78.860901,Siemens,0.188473,0.188473,297,0,297
9,0,5/20/14,1,ISMD,ISMD1,78.443268,Siemens,0.082451,0.082451,297,0,297


In [3]:
data.corr()

,Time,Visit,tSNR_raw,FD,FD_scrubbed,Total_volume,Volumes_scrubbed,Volumes_remaining
Time,1.000000,0.234910,0.040466,0.011392,0.069032,-0.588457,-0.177392,-0.150618
Visit,0.234910,1.000000,-0.251056,0.124680,0.099235,0.413924,0.113258,0.117026
tSNR_raw,0.040466,-0.251056,1.000000,0.429985,0.414551,0.001356,0.342896,-0.329023
FD,0.011392,0.124680,0.429985,1.000000,0.968295,0.199072,0.860974,-0.719334
FD_scrubbed,0.069032,0.099235,0.414551,0.968295,1.000000,0.187300,0.729764,-0.599575
Total_volume,-0.588457,0.413924,0.001356,0.199072,0.187300,1.000000,0.212241,0.341750
Volumes_scrubbed,-0.177392,0.113258,0.342896,0.860974,0.729764,0.212241,1.000000,-0.845847
Volumes_remaining,-0.150618,0.117026,-0.329023,-0.719334,-0.599575,0.341750,-0.845847,1.000000


# Extract predictive variables

The time:

In [4]:
Xt = preprocessing.scale(data.Time, with_std=False) / 365
Xt = Xt.reshape([Xt.size,1])

The scanner makes:

In [5]:
Xscan = np.zeros([data.shape[0] , 2])
Xscan[:,0] = data.Scanner == "Siemens"
Xscan[:,1] = data.Scanner == "GE"

Assemble the full model:

In [6]:
X = np.concatenate([Xt , Xscan], axis=1)
X = sm.add_constant(X)

# Relationship with FD

In [7]:
Y = preprocessing.scale(data.FD_scrubbed, with_std=True)
Y = Y.reshape([Y.size,1])
results = sm.OLS(Y, X).fit()
labels = ['Philips', 'time', 'Siemens_minus_Philips', 'GE_minus_Philips']
results.summary(xname=labels)

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.751
Model:                            OLS   Adj. R-squared:                  0.714
Method:                 Least Squares   F-statistic:                     20.12
Date:                Tue, 21 May 2019   Prob (F-statistic):           2.97e-06
Time:                        21:37:02   Log-Likelihood:                -17.364
No. Observations:                  24   AIC:                             42.73
Df Residuals:                      20   BIC:                             47.44
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Philips                   1.1398      0.186      6.138      0.000       0.752       1.527
time                      0.2195      0.106      2.064      0.052      -0.002       0.441
Siemens_minus_Philips    -1.8427      0.240     -7.691      0.000      -2.343      -1.343
GE_minus_Philips         -1.7000      0.473     -3.592      0.002      -2.687      -0.713
==============================================================================
Omnibus:                        3.898   Durbin-Watson:                   2.575
Prob(Omnibus):                  0.142   Jarque-Bera (JB):                2.725
Skew:                           0.824   Prob(JB):                        0.256
Kurtosis:                       3.102   Cond. No.                         5.20
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

# Relationship with volumes_remaining

In [8]:
Y = preprocessing.scale(data.Volumes_remaining, with_std=True)
Y = Y.reshape([Y.size,1])
results = sm.OLS(Y, X).fit()
labels = ['Philips', 'time', 'Siemens_minus_Philips', 'GE_minus_Philips']
results.summary(xname=labels)

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.384
Model:                            OLS   Adj. R-squared:                  0.292
Method:                 Least Squares   F-statistic:                     4.160
Date:                Tue, 21 May 2019   Prob (F-statistic):             0.0192
Time:                        21:37:02   Log-Likelihood:                -28.236
No. Observations:                  24   AIC:                             64.47
Df Residuals:                      20   BIC:                             69.18
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Philips                  -0.6790      0.292     -2.324      0.031      -1.288      -0.070
time                     -0.1404      0.167     -0.839      0.411      -0.489       0.209
Siemens_minus_Philips     1.2311      0.377      3.267      0.004       0.445       2.017
GE_minus_Philips          0.1456      0.744      0.196      0.847      -1.407       1.698
==============================================================================
Omnibus:                        5.254   Durbin-Watson:                   1.720
Prob(Omnibus):                  0.072   Jarque-Bera (JB):                3.766
Skew:                          -0.961   Prob(JB):                        0.152
Kurtosis:                       3.275   Cond. No.                         5.20
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

# Relationship with tSNR

In [9]:
Y = preprocessing.scale(data.tSNR_raw, with_std=True)
Y = Y.reshape([Y.size,1])
results = sm.OLS(Y, X).fit()
labels = ['Philips', 'time', 'Siemens_minus_Philips', 'GE_minus_Philips']
results.summary(xname=labels)

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.312
Model:                            OLS   Adj. R-squared:                  0.209
Method:                 Least Squares   F-statistic:                     3.024
Date:                Tue, 21 May 2019   Prob (F-statistic):             0.0536
Time:                        21:37:03   Log-Likelihood:                -29.566
No. Observations:                  24   AIC:                             67.13
Df Residuals:                      20   BIC:                             71.84
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Philips                   0.7339      0.309      2.377      0.028       0.090       1.378
time                      0.1359      0.177      0.768      0.451      -0.233       0.505
Siemens_minus_Philips    -1.1898      0.398     -2.987      0.007      -2.021      -0.359
GE_minus_Philips         -1.0733      0.787     -1.364      0.188      -2.715       0.568
==============================================================================
Omnibus:                        0.468   Durbin-Watson:                   1.360
Prob(Omnibus):                  0.791   Jarque-Bera (JB):                0.151
Skew:                           0.193   Prob(JB):                        0.927
Kurtosis:                       2.955   Cond. No.                         5.20
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""